# Gramatica este totul. Corectarea erorilor gramaticle


## Introducere

Corectarea erorilor gramaticale (GEC) se ocupă de a corecta diferite tipuri de erori din text, cum ar fi erorile de ortografie, de punctuație sau gramaticale.
Un sistem GEC primește la intrare o propoziție potențial eronată și este de așteptat să o transforme în versiunea sa corectată.

Voi trebuie sa contruiți un sistem GEC adaptat pentru limba română. Printre posibilele greșeli în limba română se regăsesc: lipsa diacriticelor, greșeli ortografice de una sau două litere per cuvânt, prepoziții incorecte, dezacorduri de gen/număr/caz/persoană, cacofonii sau greșeli de punctuație.

Căteva exemple de texte cu greșeli și variantele corectate:

* Exemplu 1:

eu am duar 10 ani si stiu melodia asta de la 7 anieste preferata mea e frumoasa si eu stiu ca te chinui mult **->** Eu am doar 10 ani și știu melodia asta de la 7 ani, este preferata mea, e frumoasă și eu știu că te chinui mult.


* Exemplu 2:

Biatul acesta este special,vocea lui,cred ca toi am avut acest gand **->** Băiatul acesta este special, vocea lui, cred că toți am avut acest gând.

* Exemplu 3:

Am mai auzit melodi dar asta este cea mai bun **->** Am mai auzit melodii dar asta este cea mai bună.


## Obiectiv

Scopul este să construiți cel mai performant model de tip GEC pentru limba română, operând sub următoarele restricții:

*   Modelul trebuie sa fie de tip encoder-decoder (de exemplu, bazat pe mBART sau mT5).
*   Folosiți doar variante "base" ale modelelor (unde acestea există).
*   Nu aveți voie să folosiți date deja generate de alte entități pentru GEC sau modele deja finetunate pentru asta.
*   Pentru antrenare aveti acces **doar** la datele pe care vi le punem la dispoziție. Acestea sunt texte care provin din articole de pe Wikipedia în română și le vom considera corecte. Aveți voie să le folosiți în orice fel doriți, să le alterați în orice mod considerați benefic. Nu aveți voie să folosiți alte texte!

## Sfaturi

* Pornind de la date curate, încercați să vă generați automat date de antrenare.
* Evaluarea se va face pe un set divers de propoziții, dar la nivel de propoziție. Astfel, la evaluare va trebui ca modelul vostru să corecteze câte o propoziție pe rând.



## Livrabile

Trebuie să submiteți următoarele:

*   Un model încărcat pe Huggingface Hub (vezi parametrul push_to_hub; alternativ puteți încărca modelul direct de pe Huggingface, din browser).
*   Un raport tehnic de maxim două pagini în care să explicați cum ați rezolvat problema. Raportul poate fi scris în engleza sau în română.


## Cerințe preliminare



### Configurație HuggingFace

Înainte de a incepe propriu-zis rezolvarea problemei trebuie să:

1. Intrați pe pagina de [HuggingFace](https://huggingface.co/Olimpiada-AI) și cereți acces la date.

2. În setări, creați Access Tokens, unul pentru "read" și unul pentru "write" și salvați-le în [Colab Secrets](https://www.youtube.com/watch?v=q87i2LZbbPc) ca `hf_read` și `hf_write`.

In [ ]:
# from google.colab import userdata

# read_access_token = userdata.get('hf_read')
# write_access_token = userdata.get('hf_write')

# mt5 requires at least 7.25gb of vram in bfloat16

### Module necesare

In [2]:
import importlib
import torch, transformers


# if '2.3.0' not in torch.__version__:
#   !pip install torch==2.3.0
# if transformers.__version__!='4.41.2':
#   !pip install transformers==4.41.2

# if importlib.util.find_spec('datasets') is None:
#   !pip install datasets==2.18.0
#   !pip install evaluate==0.4.2
#   !pip install accelerate -U

# !pip install rouge_score

Dacă tocmai ați instalat `accelerate`, executați `Runtime > Restart session and run all` din meniul Colab.

# Date

In [3]:
def alter_sentence(example):
  original_sentence = example["part"]
  altered_sentence = "A " + original_sentence
  return {"original_sentence": original_sentence, "altered_sentence": altered_sentence}

In [4]:
import textwrap

def split_paragraph(example):
  return {"part": [part for foo in example["page"] for part in textwrap.wrap(foo, 100)]}

In [5]:
# load the data

from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets

wiki_dataset = load_dataset('Olimpiada-AI/ro_wiki')

# merge splits into a single dataset
wiki_dataset = concatenate_datasets([wiki_dataset["validation"], wiki_dataset["test"]])

# split into smaller chunks
wiki_dataset = wiki_dataset.map(split_paragraph, batched=True, remove_columns="page")

# alter sentences
wiki_dataset = wiki_dataset.map(alter_sentence, batched=False, remove_columns="part")

# split into train and validation
wiki_dataset= wiki_dataset.train_test_split(test_size=0.05)

# Baseline

In [6]:
# load the pre-trained tokenizer and use it to process the data

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding, DataCollatorForSeq2Seq

tokenizer = AutoTokenizer.from_pretrained("google/mt5-base", use_fast=False)


max_length = 256

train_data = wiki_dataset['train'].map(lambda x: {'input_ids': tokenizer(x['altered_sentence'], truncation=True, max_length=max_length)["input_ids"], 'label': tokenizer(x['original_sentence'], truncation=True, max_length=max_length)["input_ids"]}, remove_columns=["original_sentence", "altered_sentence"])
val_data = wiki_dataset['test'].map(lambda x: {'input_ids': tokenizer(x['altered_sentence'], truncation=True, max_length=max_length)["input_ids"], 'label': tokenizer(x['original_sentence'], truncation=True, max_length=max_length)["input_ids"]}, remove_columns=["original_sentence", "altered_sentence"])

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/38610 [00:00<?, ? examples/s]

Map:   0%|          | 0/2033 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments

model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")

In [8]:
# define the evaluation metric

import evaluate
import numpy as np
import sys

bleu = evaluate.load("bleu")
meteor = evaluate.load('meteor')
rouge = evaluate.load('rouge')


def compute_metrics(eval_pred):
    raw_predictions, raw_labels = eval_pred
    predictions = []
    labels = []

    for pred in raw_predictions:
      pred = list(filter(lambda x: x != -100, pred))
      text_predictions = tokenizer.decode(pred, skip_special_tokens=True)
      predictions.append(text_predictions)

    for label in raw_labels:
      label = list(filter(lambda x: x != -100, label))
      text_labels = tokenizer.decode(label, skip_special_tokens=True)
      labels.append([text_labels])

    res_bleu = bleu.compute(predictions=predictions, references=labels)["bleu"]
    res_meteor = meteor.compute(predictions=predictions, references=labels)["meteor"]
    res_rouge = rouge.compute(predictions=predictions, references=labels)["rougeL"]
    return {"bleu": res_bleu, "meteor": res_meteor, "rouge-L": res_rouge}



[nltk_data] Downloading package wordnet to /home/stefan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/stefan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/stefan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# define the model and the training configuration

from transformers import AutoModelForSequenceClassification, TrainingArguments, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments

batch_size = 2
model.generation_config.max_new_tokens = 256

training_args = Seq2SeqTrainingArguments(
    output_dir="baseline_model",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    do_train=True,
    do_eval=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=1,
    logging_steps = 1,
    learning_rate=5e-5,
    warmup_steps=1,
    overwrite_output_dir=True,
    save_total_limit=3,
    bf16=True,
    load_best_model_at_end=True,
    push_to_hub=False, #
    # hub_strategy="checkpoint",
    # hub_token=write_access_token,
    # hub_private_repo=True,
    # hub_model_id='baseline_model'
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_209886/1553604777.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [10]:
torch.cuda.empty_cache()
# execute the model training
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


OutOfMemoryError: CUDA out of memory. Tried to allocate 734.00 MiB. GPU 0 has a total capacity of 5.60 GiB of which 210.62 MiB is free. Including non-PyTorch memory, this process has 4.66 GiB memory in use. Of the allocated memory 4.40 GiB is allocated by PyTorch, and 161.27 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Inference

In [ ]:
# run the trained model on validation split
eval_out = trainer.predict(val_data)
metrics = eval_out.metrics
print(metrics)

In [ ]:
# run the trained model on custom data
test_data = [["Acest este o propizite greșita", "Aceasta este o propoziție corectă"],
             ["A Ce fdci?", "Ce faci?"],
             ["A un test scurt.", "un test scurt."]]

import pandas as pd
df = pd.DataFrame(test_data)
test_data = Dataset.from_pandas(df.rename(columns={0: "input_sentence", 1: "output_sentence"}))
test_data = test_data.map(lambda x: {'input_ids': tokenizer(x['input_sentence'], truncation=True, max_length=max_length)["input_ids"], 'label': tokenizer(x['output_sentence'], truncation=True, max_length=max_length)["input_ids"]}, remove_columns=["input_sentence", "output_sentence"])

eval_out = trainer.predict(test_data)
metrics = eval_out.metrics

# nice print
import pandas as pd
print(pd.DataFrame(metrics))# run the trained model on custom data
test_data = [["Acest este o propizite greșita", "Aceasta este o propoziție corectă"],
             ["A Ce fdci?", "Ce faci?"],
             ["A un test scurt.", "un test scurt."]]

import pandas as pd
df = pd.DataFrame(test_data)
test_data = Dataset.from_pandas(df.rename(columns={0: "input_sentence", 1: "output_sentence"}))
test_data = test_data.map(lambda x: {'input_ids': tokenizer(x['input_sentence'], truncation=True, max_length=max_length)["input_ids"], 'label': tokenizer(x['output_sentence'], truncation=True, max_length=max_length)["input_ids"]}, remove_columns=["input_sentence", "output_sentence"])

eval_out = trainer.predict(test_data)
metrics = eval_out.metrics

# nice print
import pandas as pd
print(pd.DataFrame(metrics))


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

{'test_loss': 7.872952938079834, 'test_bleu': 0.4259620694975149, 'test_meteor': 0.5668402777777778, 'test_rouge-L': 0.611111111111111, 'test_runtime': 0.5601, 'test_samples_per_second': 5.356, 'test_steps_per_second': 1.785}


# Raport de maxim 300 de cuvinte